In [1]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
import json
from itertools import chain
import numpy as np
import pandas as pd
import vigra

from neuroglancer import CoordinateSpace, LocalVolume, ImageLayer, SegmentationLayer, ManagedLayer, LocalAnnotationLayer, PointAnnotation

from neuprint import Client, fetch_roi_hierarchy
from neuclease.dvid import *
from neuclease.util import *
from neuclease.misc.ngviewer import *

import hvplot.pandas

In [3]:
from neuclease import configure_default_logging
configure_default_logging()

In [43]:
pwd

'/Users/bergs/Documents/FlyEM/mito-project/proofreading/roi-subvol-annotations'

### Initialize neuroglancer viewer

In [7]:
init_ngserver('0.0.0.0', '9989')
v = create_viewer()

In [8]:
dimensions = CoordinateSpace(
    names=['x', 'y', 'z'],
    units=['nm', 'nm', 'nm'],
    scales=[8,8,8])

with v.txn() as s:
    s.dimensions = dimensions
    s.layers['grayscale'] = ImageLayer(source='precomputed://gs://neuroglancer-janelia-flyem-hemibrain/emdata/clahe_yz/jpeg')
    s.layers['segmentation'] = SegmentationLayer(source='precomputed://gs://neuroglancer-janelia-flyem-hemibrain/v1.1/segmentation')
    s.layers['mito-mask'] = SegmentationLayer(source='brainmaps://274750196357:hemibrain:mito_20190717.46637005.raw')
    s.layers['mito-mask-tab'] = SegmentationLayer(source='brainmaps://274750196357:hemibrain:mito_20190717.46637005.secgan')
    s.layers['OLD-mito-mask'] = SegmentationLayer(source='brainmaps://274750196357:hemibrain:mito_20190717.27250582')
    s.layers['roi'] = SegmentationLayer(source='precomputed://gs://neuroglancer-janelia-flyem-hemibrain/v1.1/rois')

    s.layers['segmentation'].visible = False
    s.layers['mito-mask'].visible = False
    s.layers['mito-mask-tab'].visible = False
    s.layers['OLD-mito-mask'].visible = False
    s.layers['roi'].visible = False

In [9]:
v.get_viewer_url().replace('bergs-lm4', 'localhost')

'http://localhost:9989/v/ce19c84c7fdf1343fb6dbae25000e904943d7450/'

### Segmentation sources

In [10]:
v11 = ('emdata4.int.janelia.org:8900', '20631f94c3f446d7864bc55bf515706e')
v11_seg = (*v11, 'segmentation')

mito_mask = ('emdata4.int.janelia.org:8900', '74069d6aead7436d932449c6749e9f5a', 'mito_20190717.46637005.combined') # new mask
mito_mask_scale = 1

old_mito_mask = ('emdata4.int.janelia.org:8900', '20631f94c3f446d7864bc55bf515706e', 'mito_20190717.27250582') # old mask
old_mito_mask_scale = 0

mito_seg = ('emdata3.int.janelia.org:8900', 'd31b64ac81444923a0319961736a6c31', 'masked-mito-cc')
mito_seg_scale = 0

### Load point annotations

In [11]:
mito_point_uuids = ['1cc9', '4f27', '729d']

In [20]:
dfs = []
for uuid in tqdm_proxy(mito_point_uuids):
    # Load from dvid
    #el = fetch_all_elements('https://hemibrain-dvid2.janelia.org', uuid, 'neuroglancer_todo')
    
    # Read from cached file
    el = json.load(open(f'json/{uuid}_mito_annotation_data.json', 'r'))
    dfs.append( load_elements_as_dataframe([*chain(*el.values())]) ) 

ann_df = pd.concat(dfs, ignore_index=True, sort=True)

# Per proofreader instructions, points with comments are removed,
# unless they contain the word "cluster"
ann_df = ann_df.query('comment.isnull() or "cluster" in comment')

# Upon visual inspection, a few bookmarks clearly don't belong3
# Remove them.
ann_df = ann_df.copy()
bad_points = [[23048, 20964, 17502],
              [23037, 20779, 17530],
              [33237, 14347, 14800]]
for p in bad_points:
    ann_df = ann_df[~((ann_df[[*'xyz']].values == p).all(axis=1))]

ann_df.shape

(13335, 9)

### Load ROI names and box coordinates

In [21]:
# Extract ROI names from Erika's file
rois = [*map(lambda line: re.split('[-_]', line)[0], open('erika/prod_roi_list.txt').read().split('\n'))]

# Also get the box corners
coords = [*map(lambda line: line.split('_')[-1], open('erika/prod_roi_list.txt').read().split('\n'))]
coords_xyz = [[int(x) for x in c.split(', ')] for c in coords]
coords_zyx = np.array(coords_xyz)[:, ::-1]
boxes_zyx = np.stack([coords_zyx, coords_zyx+500],1 )

In [22]:
# Some of the roi names in that file are missing the '(R)' suffix.
# Compare the names with neuprint, and add (R) if it doesn't match already.
c = Client('neuprint.janelia.org', 'hemibrain:v1.1')
g = fetch_roi_hierarchy(mark_primary=False, format='nx')
all_rois = g.nodes

new_rois = []
for roi, box in zip(rois, boxes_zyx):
    if roi not in all_rois:
        x = box[0,2]
        if x < 25525:
            roi = roi + '(R)'
        else:
            roi = roi + '(L)'
    new_rois.append(roi)

# Check to make sure all of our ROIs have valid names now.
rois = new_rois
for roi in rois:
    fetch_info(*master, roi)

rois = sorted(set(rois))

### ROI sizes

In [23]:
# ROI voxel sizes (See TODO in next cell)
from neuclease.dvid.roi import fetch_roi_size
roi_sizes = fetch_roi_size(*master, rois, processes=1).rename('roi_size')

### Load ROI volume (with modifications)

In [ ]:
#
# This cell takes a few minutes to run!
#
import vigra
from neuclease.util import normalize_image_range

# Fetch the ROI volume and downsample from scale-5 to scale-6 for faster processsing\
hemi_box = round_box(fetch_volume_box(*v11_seg), 64)
roi_vol, roi_box, roi_overlaps = fetch_combined_roi_volume(*v11, rois, False, hemi_box // (2**5))

# TODO: This might be a faster way of computing roi sizes, since we downloaded the volume anyway...
#roi_sizes = pd.Series(roi_vol.ravel(), name='roi_label').value_counts().rename('roi_size')
#roi_sizes.index = ['<unspecified>'] + rois
#roi_sizes = roi_sizes.rename_axis('roi')
#roi_sizes[:] *= (2**5)

# Cheap downsample
roi_vol = roi_vol[::2, ::2, ::2]
roi_box = roi_box // 2

# Use a distance-transform-watershed to fill the gaps between segments
# This way, points that fall just outside of an ROI are assigned to their closest ROI.
print("computing distance transform")
roi_vol = roi_vol.astype(np.uint32)
dt = vigra.filters.distanceTransform(roi_vol, True)

print("computing watershed")
dt = normalize_image_range(dt, np.uint8)
roi_ws, num_rois = vigra.analysis.watershedsNew(dt, seeds=roi_vol)

print("Adding roi layers to neuroglancer")
roi_ws_scales = 8*np.array([64,64,64])
update_layers(v, False, 'zyx', 'nm', roi_ws_scales, 'image', roi_box[0], roi_dt=dt)
update_layers(v, False, 'zyx', 'nm', roi_ws_scales, 'segmentation', roi_box[0], roi_ws=roi_ws)
with v.txn() as s:
    s.layers['roi_dt'].visible = False
    s.layers['roi_ws'].visible = False

computing distance transform
computing watershed


### Assign points to ROIs

In [ ]:
print("Assigning ROIs to each point (from watershed)")
extract_labels_from_volume(ann_df, roi_ws, roi_box, 6, rois)
ann_df.drop(columns=['roi', 'roi_label'], errors='ignore', inplace=True)
ann_df.rename(inplace=True, columns={'label': 'roi_label', 'label_name': 'roi'})

### Initialize neuroglancer annotation layers

In [ ]:
# Create one layer per user, and give each user their own color
users = sorted(ann_df['user'].unique())
colors = ['#ffff00', '#ff00ff', '#00ffff', '#ff0000', '#00ff00', '#ffffff']
with v.txn() as s:
    for user, df in ann_df.groupby('user'):
        name = f'annotations-{user}'
        s.layers[name] = LocalAnnotationLayer(dimensions=s.dimensions)
        s.layers[name].annotationColor = colors[users.index(user)]
        for roi, point_xyz in zip(df['roi'], df[[*'xyz']].values):
            s.layers[name].annotations.append(PointAnnotation(id=repr(point_xyz), point=point_xyz))            

### Fetch mito class / mito id for each point

In [ ]:
#box_user_df['body'] = fetch_labels(*v11_seg, box_user_df[[*'zyx']].values)

ann_df['mito_mask'] = fetch_labels(*mito_mask, ann_df[[*'zyx']].values // (2**mito_mask_scale))
ann_df['mito_sv'] = fetch_labels(*mito_seg, ann_df[[*'zyx']].values // (2**mito_seg_scale), supervoxels=True)
ann_df['mito_body'] = fetch_labels(*mito_seg, ann_df[[*'zyx']].values // (2**mito_seg_scale))

#ann_df['old_mito_mask'] = fetch_labels(*old_mito_mask, ann_df[[*'zyx']].values)
#box_user_df['body'] = fetch_labels(*v11_seg, ann_df[[*'zyx']].values)
#box_user_df['body_size'] = fetch_sizes(*v11_seg, ann_df['body']).values

### Fetch small cubes around points that didn't land on the mask

In [ ]:
print("non-mask points before:", ann_df.query('mito_mask == 0 or mito_mask == 4').shape[0])

# For points which have no mask, try a little harder:
# fetch a small cube around the point.
r = 20

for row in tqdm_proxy(ann_df.itertuples(), total=len(ann_df)):
    if row.mito_mask in (0,4):
        p = np.array([row.z, row.y, row.x])
        box = np.array([p - r, p + r + 1])
        mask_labels = fetch_labelmap_voxels(*mito_mask, box // (2**mito_mask_scale))
        mask_labels = {*pd.unique(mask_labels.ravel())} - {0,4}
        if len(mask_labels):
            ann_df.loc[row.Index, 'mito_mask'] = max(mask_labels)

print("non-mask points after:", ann_df.query('mito_mask == 0 or mito_mask == 4').shape[0])

In [ ]:
ann_df['mito_size'] = fetch_sizes(*mito_seg, ann_df['mito_sv'], supervoxels=True, processes=8).values

In [ ]:
#mask_counts = ann_df['mito_mask'].value_counts().sort_index()
#old_mask_counts = ann_df['old_mito_mask'].value_counts().sort_index()

In [ ]:
#print("Mito mask misses:")
#ann_df.query('mito_mask == 0 or mito_mask == 4')[[*'xyz', 'roi', 'user', ]].head(20)

In [ ]:
#print("Mito seg misses:")
#ann_df.query('mito_seg == 0 or mito_seg == 4')[[*'xyz', 'roi', 'user']].head(20)

In [ ]:
#ann_df.query('mito_seg == 0')[[*'xyz', 'roi', 'user']]

In [ ]:
MIN_MITO_SIZE = 10_000
MIN_BODY_SIZE = 1_000_000

analysis_scale = 1

# It just so happens that none of the cubes are closer to each 
# other than ~1000 px, so a cube "radius" of 600 captures one 
# entire 500px cube no matter which point in it we start from,
#  without accidentally capturing points from nearby cubes.
NEIGHBORHOOD_RADIUS = 600

def _process_neighbors(user_df, neighbors):
    roi = neighbors['roi'].mode().iloc[0]
    
    if not (neighbors['roi'] == roi).all():
        # Oops, this box contains multiple ROIs.
        # Overwrite the roi column in the GLOBAL point list to make it simpler to analyze later.
        # (Close enough.)
        vc = neighbors['roi'].value_counts()
        print(vc[vc > 0])
        print(f"warning: box has multiple rois: {vc[vc > 0].to_dict()}")
        print(f"         overwriting all points in this box with roi {roi}")
        print(f"         example point: {neighbors[[*'xyz']].values[0].tolist()}")
        ann_df.loc[neighbors.index, 'roi'] = roi

    assert not neighbors['point_processed'].any(), \
        ("Detected overlapping neighborhood.\n"
         "Either your neighborhood radius is too small, or there\n"
         "is a point floating too far from its home cube (delete it please).")
    box = [neighbors[[*'zyx']].values.min(axis=0),
           neighbors[[*'zyx']].values.max(axis=0) + 1]

    mito_box = round_box(box, 2**mito_seg_scale) // (2**mito_seg_scale)
    mito_box //= (2**(analysis_scale-mito_seg_scale))
    mito_vol = fetch_labelmap_voxels(*mito_seg, mito_box, analysis_scale-mito_seg_scale, supervoxels=True)

    mito_vol_svs = pd.unique(mito_vol.ravel())
    mito_vol_sv_sizes = fetch_sizes(*mito_seg, mito_vol_svs, supervoxels=True).rename_axis('mito_sv').rename('mito_size')
    mito_vol_sv_sizes.loc[0] = 0
    mito_vol_sv_sizes *= (2**mito_seg_scale)

    # mask samples are just tracked by coordinate.
    mask_true_positives = neighbors.query('mito_mask != 0 and mito_mask != 4')[[*'zyx']].values
    mask_false_negatives = neighbors.query('mito_mask == 0 or mito_mask == 4')[[*'zyx']].values

    mask_recall = len(mask_true_positives) / (len(mask_true_positives) + len(mask_false_negatives))

    # seg samples are tracked by mito id
    seg_true_positives = neighbors.query('mito_sv != 0 and mito_size >= @MIN_MITO_SIZE')['mito_sv']
    seg_false_negatives = neighbors.query('mito_sv == 0 or mito_size < @MIN_MITO_SIZE')['mito_sv']
    seg_all_positives = mito_vol_sv_sizes[mito_vol_sv_sizes >= MIN_MITO_SIZE].index
    seg_false_positives = set(seg_all_positives) - set(seg_true_positives)

    seg_recall = len(seg_true_positives) / (len(seg_true_positives) + len(seg_false_negatives))
    seg_precision = len(seg_true_positives) / (len(seg_true_positives) + len(seg_false_positives))

    user_df.loc[neighbors.index, 'point_processed'] = True
    cube_id = roi + ' / ' + user.split('@')[0]
    result = (cube_id, roi, user,
              mask_recall,
              seg_recall, seg_precision,
              #mito_body_recall, mito_body_precision,
              len(mask_true_positives), len(mask_false_negatives),
              len(seg_true_positives), len(seg_false_negatives), len(seg_all_positives), len(seg_false_positives),
              #len(mito_body_true_positives), len(mito_body_false_negatives)
             )
    return result

def process_user(user_df):
    user_df = user_df.copy()
    user_df['point_processed'] = False
    user = user_df['user'].iloc[0]
    assert (user == user_df['user']).all()
    
    results = []
    remaining = user_df.loc[~user_df['point_processed']]
    prog = tqdm_proxy(total=len(remaining), leave=False)
    while len(remaining) > 0:
        r = NEIGHBORHOOD_RADIUS
        p = remaining[[*'zyx']].iloc[0].values
        try:
            neighborhood_box = np.array([p - r, p + r + 1])
            neighbor_selection = (user_df[[*'zyx']] >= neighborhood_box[0]).all(axis=1)
            neighbor_selection &= (user_df[[*'zyx']] < neighborhood_box[1]).all(axis=1)

            neighbors = user_df.loc[neighbor_selection]
            results.append( _process_neighbors(user_df, neighbors) )
            remaining = user_df.loc[~(user_df['point_processed'])]
            prog.update(len(neighbors))
        except:
            print("Failed at point:", p[::-1])
            raise

    cols = ("cube_id", "box_roi", "user",
            "mask_recall",
            "seg_recall", "seg_precision",
            #"mito_body_recall", "mito_body_precision",
            "mask_true_positives", "mask_false_negatives",
            "seg_true_positives", "seg_false_negatives", "seg_all_positives", "seg_false_positives",
            #"mito_body_true_positives", "mito_body_false_negatives"
           )

    return pd.DataFrame(results, columns=cols)

print("Processing each user")
results = []
for user, user_df in tqdm_proxy(ann_df.groupby('user'), total=ann_df['user'].nunique()):
    results.append(process_user(user_df))
cube_results_df = pd.concat(results, ignore_index=True)


In [ ]:
cube_results_df = cube_results_df.merge(roi_sizes.rename_axis('box_roi'), 'left', on='box_roi')

In [ ]:
# results_df.hvplot.scatter('mask_recall', 'seg_precision',
#                           height=1200, width=1200, xlim=[0.0, 1.05], ylim=[0.0, 1.05], grid=True,
#                           s='roi_size', scale=0.005,
#                           line_color='white', hover_cols=['box_roi'], color='box_roi', legend='right')

In [ ]:
cube_results_df.hvplot.scatter('mask_recall', 'seg_precision',
                               height=600, width=600, xlim=[0.8, 1.05], ylim=[0.0, 1.05], grid=True,
                               s='roi_size', scale=0.005,
                               line_color='white', hover_cols=['box_roi'], color='box_roi'
                               #, legend='right')
                              )

In [ ]:
avg_results_df = (cube_results_df.groupby(['box_roi'])
                                .agg({'mask_recall': 'mean', 'seg_recall': 'mean', 'seg_precision': 'mean', 'roi_size': 'first'})
                                .rename_axis('roi').reset_index())
avg_results_df = avg_results_df.sort_values('roi_size', ascending=False)
avg_results_df.hvplot.scatter('mask_recall', 'seg_precision',
                              height=500, width=500, xlim=[0.8, 1.02], ylim=[0.0, 1.05], grid=True,
                              s='roi_size', scale=0.005, line_color='white', hover_cols=['roi'])

In [ ]:
#mean_coords = ann_df.groupby('roi')[[*'xyz']].mean().fillna(0).astype(int)
#d = avg_results_df.merge(mean_coords, 'left', left_on='roi', right_index=True)
#print(d.sort_values('seg_precision').to_csv(header=True, index=False))

In [41]:
from io import StringIO
import yaml
cfg = yaml.load("""\
  zarr:
    path: /nrs/flyem/bergs/hemibrain-mito-masks/hemibrain-v1.1-masked-by-mito.zarr
    dataset: s1
""")

/opt/miniconda/envs/flyem/lib/python3.7/site-packages/ipykernel_launcher.py:7: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  import sys


In [42]:
from flyemflows.volumes import VolumeService
vs = VolumeService.create_from_config(cfg)

RuntimeError: File does not exist: /nrs/flyem/bergs/hemibrain-mito-masks/hemibrain-v1.1-masked-by-mito.zarr
You did not specify 'create-if-necessary' in the config, so I won't create it.:
